# Setup

In [ ]:
pip install ortools

In [1]:
from ortools.linear_solver import pywraplp

# Linear programming example: Maximise revision timetables

It's quite easy to set up a linear programming solution to maximise timetable generation (though here I just generate numbers of hours revised rather than when each subject is revised). The most important thing is to make sure that the constraints are defined properly, since they govern the entire solution.

In [18]:
def maximise():
    # Number of hours of revision that the user wants to do.
    revision_hrs= 10

    # All the subjects that the user takes.
    subjs = ["Maths", "English", "French", "Biology", "Physics", "Computing"]

    # Accumulated from exam results and previous timetable history - the mean
    # increase in grade per hour of revision, specific to each subject that the
    # user takes.
    grade_rates = [0.2, 0.12, 0.3, 0.23, 0.08, 0.43]

    # One-hot array of the subjects that the user wants to revise for.
    subjs_revised = [1, 1, 0, 0, 1, 0]

    # Target grade increases (i.e. how many grades up is the user aiming for).
    grade_targets = [1, 3, 2, 2, 4, 1]

    # Instantiate a Glop solver and give it a name.
    solver = pywraplp.Solver("MaximiseSolver",
                             pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    
    # Define objective and create variables.
    objective = solver.Objective()
    revision_plan = [None] * len(subjs)

    for i in range(0, len(subjs)):
        # Create a new variable with a range from 0 to infinity and set its
        # name as the name of the subject.
        revision_plan[i] = solver.NumVar(0.0, solver.infinity(), subjs[i])

        # Weight each subject based on whether the grade rates. If the user has
        # not chosen to revise a subject, the weighting is 0.
        weighting = grade_rates[i] * subjs_revised[i] * grade_targets[i]
        objective.SetCoefficient(revision_plan[i], weighting)

    # Haha. Maximise. We want the biggest positive changes in grades, so we 
    # want the objective to be maximization.
    objective.SetMaximization()

    # Set up the constraints (maximum revision hours).
    hrs_constraint = solver.Constraint(0.0, revision_hrs)
    for i in range(0, len(subjs)):
        # Weird one to explain - time doesn't tick differently for different
        # subjects, so spending one hour doing maths is just as costly in terms
        # of time as spending one hour doing physics. Therefore we set the
        # constraint coefficient to 1 for each subject.
        hrs_constraint.SetCoefficient(revision_plan[i], 1)

    # Constraint to ensure that all chosen revision subjects actually get
    # revised for at least one hour, but not for more than half the revision
    # time.
    subj_constraints = [None] * len(subjs)
    for i in range(0, len(subj_constraints)):
        # Each subject gets its own constraint.
        subj_constraints[i] = solver.Constraint(subjs_revised[i], 
                                                revision_hrs / 2)
        subj_constraints[i].SetCoefficient(revision_plan[i], 1)

    # Solve the problem.
    status = solver.Solve()

    # If the solver was able to find an optimal solution. Sometimes no solution
    # can be found, and at other times, a suboptimal solution is found.
    if status == solver.OPTIMAL:
        total_hrs = 0

        # Show number of hours for each subject.
        for (i, subj) in enumerate(revision_plan):
            subj_hrs = subj.solution_value()
            if subj_hrs > 0:
                print(f"Revise {subj_hrs} hours of {subjs[i]}")
                total_hrs += subj_hrs

        # Total number of hours should just be as specified by the user.
        print(f"{total_hrs} hours of revision in total!")

In [19]:
maximise()

Revise 1.0 hours of Maths
Revise 5.0 hours of English
Revise 4.0 hours of Physics
10.0 hours of revision in total!
